In [12]:
import time
import boto3


In [13]:

client = boto3.client("elasticbeanstalk")

s3_bucket_name = 'peakyblinders1'
security_group_name = 'web-tier'
instance_key_name = 'Vishal'
ec2_role_name = 'Devil'      # s3 Access Role


In [14]:

client.create_application_version(
    ApplicationName="portfolioapp",
    AutoCreateApplication=True,
    Description="vishal_prajapati portfolio website",
    Process=True,
    SourceBundle={
        "S3Bucket": "peakyblinders1",
        "S3Key": "NewPortfolio.zip",
    },
    VersionLabel="v1",
)

while True:
    response = client.describe_application_versions(
        ApplicationName="portfolioapp",
        VersionLabels=[
            "v1",
        ],
        MaxRecords=1,
    )
    if response["ApplicationVersions"][0]["Status"] != "PROCESSED":
        print("Trying to create Application...")
        time.sleep(5)
    else:
        print("Succesfully created Application...")
        break

response = client.create_environment(
    ApplicationName="portfolioapp",
    CNAMEPrefix="vishalprajapati",
    EnvironmentName="portfolio-env",
    SolutionStackName="64bit Amazon Linux 2 v5.8.5 running Node.js 18",
    VersionLabel="v1",
    OptionSettings=[
        {
            "Namespace": "aws:autoscaling:launchconfiguration",
            "OptionName": "IamInstanceProfile",
            "Value": "Devil",
        },
        {
            "Namespace": "aws:autoscaling:launchconfiguration",
            "OptionName": "InstanceType",
            "Value": "t2.micro",
        },
        {
            "Namespace": "aws:autoscaling:launchconfiguration",
            "OptionName": "EC2KeyName",
            "Value": "Vishal",
        },
        {
            "Namespace": "aws:autoscaling:launchconfiguration",
            "OptionName": "ImageId",
            "Value": "ami-097e7acc1d0c8b1b9",
        },
        {
            "Namespace": "aws:autoscaling:launchconfiguration",
            "OptionName": "SecurityGroups",
            "Value": "sg-06a3c78a81d0056da",
        },
        {
            "Namespace": "aws:autoscaling:trigger",
            "OptionName": "Statistic",
            "Value": "Average",
        },
        {
            "Namespace": "aws:autoscaling:trigger",
            "OptionName": "Period",
            "Value": "1",
        },
        {
            "Namespace": "aws:autoscaling:trigger",
            "OptionName": "UpperThreshold",
            "Value": "80",
        },
        {
            "Namespace": "aws:autoscaling:trigger",
            "OptionName": "UpperBreachScaleIncrement",
            "Value": "1",
        },
        {
            "Namespace": "aws:autoscaling:trigger",
            "OptionName": "MeasureName",
            "Value": "CPUUtilization",
        },
        {
            "Namespace": "aws:autoscaling:trigger",
            "OptionName": "LowerThreshold",
            "Value": "20",
        },
        {
            "Namespace": "aws:autoscaling:trigger",
            "OptionName": "LowerBreachScaleIncrement",
            "Value": "-1",
        },
        {
            "Namespace": "aws:autoscaling:asg",
            "OptionName": "Availability Zones",
            "Value": "Any 2",
        },
        {
            "Namespace": "aws:autoscaling:asg",
            "OptionName": "MaxSize",
            "Value": "3",
        },
        {
            "Namespace": "aws:autoscaling:asg",
            "OptionName": "MinSize",
            "Value": "1",
        },
        {
            "Namespace": "aws:ec2:vpc",
            "OptionName": "Subnets",
            "Value": "subnet-06d8d87920e940944"
        },
    ],
)

print("Envoirnment Created...")


Trying to create Application...
Succesfully created Application...
Envoirnment Created...


In [15]:
cloudfront_client = boto3.client("cloudfront", region_name="ap-south-1")

cloudfront_client.create_distribution(
    DistributionConfig={
        "CallerReference": "09-09-2023",
        'Origins': {
            'Quantity': 1,
            'Items': [
                {
                    'Id': 'peakyblinders1-origin',
                    'DomainName': 'peakyblinders1.s3.ap-south-1.amazonaws.com',
                    'S3OriginConfig': {
                        'OriginAccessIdentity': ''
                    },
                },
            ]
        },
        'DefaultCacheBehavior': {
            'TargetOriginId': 'peakyblinders1-origin',
            'ViewerProtocolPolicy': 'allow-all',
            'ForwardedValues': {
                'QueryString': False,
                'Cookies': {
                    'Forward': 'all',
                },
                'Headers': {
                    'Quantity': 0,
                },
                'QueryStringCacheKeys': {
                    'Quantity': 0,
                }
            },
            'MinTTL': 1000,
        },
        "Enabled": True,
        "Comment": "distribution to host portfolio",
    }
)


DistributionAlreadyExists: An error occurred (DistributionAlreadyExists) when calling the CreateDistribution operation: The caller reference that you are using to create a distribution is associated with another distribution. Already exists: E1SKXYD0187KAV